In [ ]:
#pip install psycopg2

  Using cached psycopg2-2.9.10-cp311-cp311-win_amd64.whl.metadata (5.0 kB)
Using cached psycopg2-2.9.10-cp311-cp311-win_amd64.whl (1.2 MB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import psycopg2

# Настройки подключения к БД
DB_CONFIG = {
    "dbname": "email_ai",
    "user": "postgres",  # Замени на свой логин
    "password": "admin",  # Укажи свой пароль
    "host": "localhost",
    "port": "5432",
}

def get_db_connection():
    """Создает подключение к базе данных"""
    return psycopg2.connect(**DB_CONFIG)

def fetch_bosses():
    """Получает список боссов из базы"""
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM bosses;")
    bosses = [row[0] for row in cursor.fetchall()]
    conn.close()
    return bosses

def fetch_trigger_words():
    """Получает список триггерных слов из базы"""
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT word FROM trigger_words;")
    words = [row[0] for row in cursor.fetchall()]
    conn.close()
    return words

def fetch_prompt(label):
    """Получает соответствующий промпт по метке"""
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT prompt FROM prompts WHERE label = %s;", (label,))
    result = cursor.fetchone()
    conn.close()
    return result[0] if result else ""


In [ ]:
#pip install langchain-gigachat

  Using cached langchain_gigachat-0.3.8-py3-none-any.whl.metadata (2.9 kB)
  Using cached gigachat-0.1.39.post1-py3-none-any.whl.metadata (14 kB)
  Using cached types_requests-2.32.0.20250306-py3-none-any.whl.metadata (2.3 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached pydantic-2.10.6-py3-none-any.whl.metadata (30 kB)
  Using cached langsmith-0.3.18-py3-none-any.whl.metadata (15 kB)
  Using cached tenacity-9.0.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-win_amd64.whl.metadata (2.1 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached anyio-4.9.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached certifi-2025.1.31-py3-none-any.whl.metadata (2.5 kB)
  Using cached httpcore-1.0.7-py3-none-any.whl.metadata (21 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_gigachat.chat_models import GigaChat
from langchain_core.prompts import PromptTemplate


giga = GigaChat(
    # Для авторизации запросов используйте ключ, полученный в проекте GigaChat API
    credentials="YjBiMGE1YWQtMjNiMS00YTI0LTg4MGItN2NmYjYzNTE5M2RhOjdmN2U5MzRkLTQ2YzYtNDNmYy05MTQ4LTI0NjM4MjFjOGQ0Mw====",
    verify_ssl_certs=False,
)

# Списки боссов и триггерных слов
BOSSES = ["Фишер", "Ларионов", "Смольянинова", "Греф"]
TRIGGER_WORDS = ["инцидент", "сбой", "авария", "блокировка"]

instruction="1. если отключаем боевой сервер - предварительно уведомляем сотрудников по почте. 2. отключения можно производить строго с 18:00 до 06:00. 3. кратко опишите возникшую проблему простым языком. 4.придерживайтесь принципа деловой переписки"

# Функция для чтения письма из файла
def read_email_from_file(file_path: str):
    """Считывает тему, отправителя и текст письма из файла."""
    with open(file_path, "r", encoding="utf-8") as file:
        lines = file.readlines()
    sender = lines[0].strip()
    subject = lines[2].strip()
    content = " ".join(line.strip() for line in lines[4:])
    return subject, sender, content

# Функция для анализа важности письма
def analyze_email(subject: str, sender: str, content: str):
    """Анализ письма с учетом данных из БД"""
    BOSSES = fetch_bosses()
    TRIGGER_WORDS = fetch_trigger_words()

    sender_boss = sender in BOSSES
    trigger_word = any(word in content.lower() for word in TRIGGER_WORDS)

    # Определяем тип письма
    if sender_boss and trigger_word:
        label = "mail_boss_and_trigger"
    elif sender_boss:
        label = "mail_boss"
    elif trigger_word:
        label = "mail_trigger"
    else:
        label = "mail_simple"

    prompt_text = fetch_prompt(label)

    if prompt_text:
        response = giga([HumanMessage(content=prompt_text)])
        return f"Маркировка: {label}\nОтвет GigaChat: {response.content}"
    else:
        return f"Маркировка: {label} (Нет необходимости в ответе)"


In [ ]:
# Вебхук для бота
@app.route("/webhook", methods=["POST"])
def webhook():
    data = request.json

    sender = data.get("sender")
    content = data.get("message")

    response_text = analyze_email(sender, content)

    return jsonify({"reply": response_text})

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)

Тест-кейс 1

"Иванов

Обед

Привет, ты на обед в 2 или в 3 пойдешь?
Еда в столовой — это всегда интересное сочетание вкусов и ароматов. Здесь можно попробовать разнообразные блюда, от классических супов до сытных гарниров и десертов. В обеденное время столовая наполняется аппетитными запахами свежесваренных борщей, жареной картошки и ароматной выпечки. Каждый день меню обновляется, предлагая новые варианты для любителей разнообразия."

In [19]:
file_path = "email_test_v1/email1.txt"  # Укажите путь к файлу с письмом
subject, sender, content = read_email_from_file(file_path)
result = analyze_email(subject, sender, content)
print("Результат анализа:", result)

Результат анализа: Маркировка: mail_simple (Нет необходимости в ответе)


Тест-кейс 2

"Сидоров

Тренажерный зал

Привет.
Когда с нами начнешь ходить?
Тренажерный зал оставил приятное впечатление благодаря современному оборудованию и просторным помещениям. Персонал вежливый и всегда готов помочь с настройкой тренажеров или советом по тренировкам. Особенно порадовала зона кардио с новыми беговыми дорожками и велотренажерами. Чистота и порядок также на высоком уровне, что делает тренировки комфортными и приятными."

In [20]:
file_path = "email_test_v1/email2.txt"  # Укажите путь к файлу с письмом
subject, sender, content = read_email_from_file(file_path)
result = analyze_email(subject, sender, content)
print("Результат анализа:", result)

Результат анализа: Маркировка: mail_simple (Нет необходимости в ответе)


Тест-кейс 3

"Греф

Человекоцентричность как миссия

Друзья, помним: Человекоцентричность в Сбере проявляется в приоритете интересов клиентов и сотрудников. Компания стремится создавать продукты и услуги, которые делают жизнь людей проще и удобнее. Важную роль играет забота о персонале, развитие корпоративной культуры и поддержка инициатив, направленных на улучшение условий труда."

In [21]:
file_path = "email_test_v1/email3.txt"  # Укажите путь к файлу с письмом
subject, sender, content = read_email_from_file(file_path)
result = analyze_email(subject, sender, content)
print("Результат анализа:", result)

Результат анализа: Маркировка: mail_boss
Ответ GigaChat: Ваше сообщение выглядит так:

**Тема: Важная задача**

Уважаемый [Имя Отчество]!

Прошу Вас незамедлительно приступить к выполнению важной задачи. Ответственность за выполнение этой задачи лежит полностью на Вас. Вам необходимо собрать всю необходимую информацию и представить отчет по текущему проекту XYZ до конца недели. 

Пожалуйста, предоставьте мне план действий и сроки выполнения задачи как можно скорее.

С уважением,  
[Ваша должность]  
[Ваше имя]


Тест-кейс 4

"Джейсонов

В серверной авария!

Авария в серверной произошла из-за затопления, вызванного прорывом водопроводной трубы. Вода повредила оборудование, что привело к остановке работы серверов и утрате части данных. Специалисты оперативно устранили причину затопления и начали восстановление системы, однако полное возвращение к нормальной работе потребует времени."

In [22]:
file_path = "email_test_v1/email4.txt"  # Укажите путь к файлу с письмом
subject, sender, content = read_email_from_file(file_path)
result = analyze_email(subject, sender, content)
print("Результат анализа:\n", result)

Результат анализа:
 Маркировка: mail_trigger
Ответ GigaChat: **Уведомление о сбое системы**

Ваша система столкнулась со следующей проблемой: *Ошибка при выполнении операции*. Пожалуйста, подтвердите эту ошибку перед продолжением.

### Возможные причины:
1. Недостаточно ресурсов для выполнения задачи.
2. Ошибка в коде программы или операционной системе.
3. Проблемы с сетевым подключением или оборудованием.
4. Временная ошибка сервера или базы данных.

### Рекомендации по устранению ошибки:

1. **Проверьте ресурсы системы**:
   - Закройте ненужные приложения и процессы.
   - Увеличьте объем памяти или дискового пространства, если это возможно.
   
2. **Обновите систему и программное обеспечение**:
   - Проверьте наличие обновлений для операционной системы и приложений.
   - Обновите драйверы оборудования до последних версий.

3. **Перезагрузите систему**:
   - Это может решить временные сбои и оптимизировать работу системы.

4. **Проведите диагностику сети**:
   - Переподключитесь к сет

Тест-кейс 5

"Фишер

Обнаружен сбой на ИФТ

Молодой стажер Михаил случайно удалил рабочую базу данных, перепутав её с тестовой. Поняв свою ошибку, он попытался всё исправить, но данные были утрачены. Начальник отдела вызвал его на разговор, но, к удивлению Михаила, оказался понимающим. Коллеги восстановили данные из резервных копий, а сам инцидент стал ценным уроком для стажера."

In [23]:
file_path = "email_test_v1/email5.txt"  # Укажите путь к файлу с письмом
subject, sender, content = read_email_from_file(file_path)
result = analyze_email(subject, sender, content)
print("Результат анализа:", result)

Результат анализа: Маркировка: mail_boss_and_trigger
Ответ GigaChat: **Уведомление о сбое системы**

**Краткое описание проблемы:**
Не удалось загрузить основную библиотеку приложения `libstdc++`, что привело к остановке работы программы.

**Предлагаемое решение:**
1. Убедитесь, что у вас установлена версия дистрибутива операционной системы, поддерживающая запуск данной версии программы.
2. Проверьте наличие всех необходимых пакетов для вашей операционной системы. Если они отсутствуют, установите их вручную через менеджеры пакетов.
3. Перезагрузите систему и попробуйте снова запустить программу.
4. Если проблема сохраняется, обновите саму программу до последней версии.
5. В случае неудачи обратитесь к администратору системы или разработчикам программы для получения дополнительной помощи.

---

Вот письмо, которое я взял из текстового файла:

---

**Текст письма:**

```
Уважаемый пользователь!

Сообщаем вам, что произошла ошибка при запуске программы. Не удалось загрузить основную библи